In [73]:
from datetime import datetime
from datetime import date
import requests
import pandas as pd
import json
import hvplot
import hvplot.pandas
import re
from sklearn import preprocessing 
from textblob import TextBlob
import time
import numpy as np


In [2]:
# load Twitter API keys

import os
import dotenv
import tweepy as tw

#dotenv.load_dotenv('.env')

from dotenv import load_dotenv #used examples from API pulls in class exercises
load_dotenv()

consumer_key = os.getenv('TWITTER_CONSUMER_KEY')
consumer_secret = os.getenv('TWITTER_CONSUMER_SECRET')
access_token = os.getenv('TWITTER_ACCESS_TOKEN')
access_token_secret = os.getenv('TWITTER_ACCESS_TOKEN_SECRET')


In [3]:
#authorize Twitter API

auth = tw.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tw.API(auth, wait_on_rate_limit=True)



In [240]:
# Read the CSV located at the file path into a Pandas DataFrame

btc_df = pd.read_csv("BTC_USD2.csv", parse_dates=True, infer_datetime_format=True)
btc_df['Date'] =  pd.to_datetime(btc_df['Date'])
btc_df = btc_df [["Date","Price"]]
btc_df = btc_df.rename(columns={'Price': 'Close'})
btc_df['Close']=btc_df['Close'].replace(',', '', regex=True).astype(float)
btc_df = btc_df.set_index('Date')
btc_df = btc_df.dropna()

# Print the DataFrame
btc_df

,Close
Date,
2018-01-18,11045.00
2018-01-19,11476.00
2018-01-20,12728.00
2018-01-21,11514.00
2018-01-22,10771.00
...,...
2021-01-19,35958.00
2021-01-20,35521.10
2021-01-21,30873.00


In [5]:
# DEFINING A FUNCTION TO REMOVE REMOVE URL FOR ALL TWEETS TO USE LATER ON

#def remove_url(txt):
#    return " ".join(re.sub("([^0-9A-Za-z \t])|(\w+:\/\/\S+)", "", txt).split())


def remove_pattern(input_txt, pattern):
    r = re.findall(pattern, input_txt)
    for i in r:
        input_txt = re.sub(i, '', input_txt)        
    return input_txt

def clean_tweets(tweets):
    #remove twitter Return handles (RT @xxx:)
    tweets = np.vectorize(remove_pattern)(tweets, "RT @[\w]*:") 
    
    #remove twitter handles (@xxx)
    tweets = np.vectorize(remove_pattern)(tweets, "@[\w]*")
    
    #remove URL links (httpxxx)
    tweets = np.vectorize(remove_pattern)(tweets, "https?://[A-Za-z0-9./]*")
    
    #remove special characters, numbers, punctuations (except for #)
    tweets = np.core.defchararray.replace(tweets, "[^a-zA-Z]", " ")
    
    return tweets

In [6]:
# SECTION A: GETTING A ALL TWEETS FROM @SatoshiLite (CHARLIE LEE CREATOR OF LITECOIN 841K FOLLOWERS)

screen_name = 'SatoshiLite'

charlie_tweets = tw.Cursor(api.user_timeline,
              screen_name=screen_name,
              tweet_mode='extended',
              lang =" en").items()

charlie_json = []

#for tweet in crypto_tweets:
#    if (not tweet.retweeted) and ('RT @' not in tweet.full_text):
#        crypto_json.append(tweet._json)
 
for tweet in charlie_tweets:
        charlie_json.append(tweet._json)



In [15]:
# writing a list of items we need from the json file and creating DataFrame for specific tweet columns

charlie_list=[]

with open('charlie.txt', 'w') as file:
        file.write(json.dumps(charlie_json, indent=4))

with open('charlie.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        text = clean_tweets(each_dictionary['full_text'])
        Date = each_dictionary['created_at']
        name = each_dictionary['user']['screen_name']
        
        charlie_list.append({'tweet_id': str(tweet_id),
                             'text': str(text),
                             'name': str(name),
                             'Date': Date,
                            })

        charlie_df = pd.DataFrame(charlie_list, columns = 
                                  ['Date','tweet_id', 'text', 
                                   'name'])
        
charlie_df.to_csv('charlie.csv', index = False)
        
#pd.options.display.max_columns = None
#pd.options.display.max_colwidth = 500


In [95]:
#Reading in CSV file
#(Note I had to export into csv so those without the twitter API can use the formatted data in csv format)

pd.options.display.max_colwidth = 500
charlie_df = pd.read_csv('charlie.csv')
charlie_df

,Date,tweet_id,text,name
0,Fri Jan 22 04:48:09 +0000 2021,1352478133224493059,"A more detailed explanation of the whole \n\n""A double-spend broke Bitcoin"" FUD that was circulated by an irresponsible publica…",SatoshiLite
1,Thu Jan 21 21:17:00 +0000 2021,1352364596871397377,,SatoshiLite
2,Wed Jan 20 08:18:53 +0000 2021,1351806388880646145,"3 years, 42 episodes, and a lot of corny jokes later, is over! It was an honor and a blast doing the show wit…",SatoshiLite
3,Wed Jan 20 00:32:40 +0000 2021,1351689062436925440,"After over 3 years of dad jokes, being late, not speaking English, &amp; selling at ATH, we…",SatoshiLite
4,Tue Jan 19 20:02:38 +0000 2021,1351621106268991488,Supposed to be going live in a few minutes except is late again:)\n\nLink: …,SatoshiLite
...,...,...,...,...
3188,Mon Jul 02 04:41:30 +0000 2018,1013643806971817984,Only with money you can live without for years. 😀,SatoshiLite
3189,Mon Jul 02 03:38:50 +0000 2018,1013628034891448321,Yes,SatoshiLite
3190,Mon Jul 02 03:28:02 +0000 2018,1013625316470710272,"No, not really. But I heard he's the One. #matrix",SatoshiLite
3191,Mon Jul 02 03:22:21 +0000 2018,1013623888335462401,At the Asia Blockchain Summit with . He's about to go on stage for his keynote. I wonder if he will say he has more money than Taiwan and won't bring his technology to Taiwan. 😂,SatoshiLite


In [100]:
# Cleaning up Date from twitter format to python readable format

charlie_df = charlie_df.dropna().drop_duplicates()
charlie_df['Date'] = pd.to_datetime(charlie_df['Date'],utc=True)
charlie_df['Date'] = charlie_df['Date'].dt.date
charlie_df['Date'] = pd.to_datetime(charlie_df['Date'])
charlie_df.dtypes

# Adding textblob sentiment
charlie_df['textblob']= charlie_df['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
charlie_df

# Adding vader sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()

def sentiment_score_compound(sentence):
    score = analyzer.polarity_scores(sentence)
    return score['compound']

charlie_df['vader_compound']= charlie_df.apply(lambda row: sentiment_score_compound(row['text']), axis=1)
charlie_df.drop_duplicates()

# Only getting tweets with word bitcoin
search = ['Bitcoin','bitcoin','BTC','btc','crypto','cryptocurrency']

charlie_df = charlie_df[charlie_df['text'].str.contains('|'.join(search))]
charlie_df

,Date,tweet_id,text,name,textblob,vader_compound
0,2021-01-22,1352478133224493059,"A more detailed explanation of the whole \n\n""A double-spend broke Bitcoin"" FUD that was circulated by an irresponsible publica…",SatoshiLite,0.366667,-0.8192
9,2021-01-15,1349895648686948353,Here's how I got accidentally Bitcoin rich...\n\nWho wants some?,SatoshiLite,0.287500,0.2960
10,2021-01-14,1349823678008102915,"“The Classic Bitcoiner Response”, featuring",SatoshiLite,0.166667,0.0000
22,2021-01-07,1346987511478849538,Excited to have $4B mcap $WBTC and $WETH launching on #TRON with \n\nMeanwhile Wrapped #Litecoin $WLTC on #TRON is…,SatoshiLite,0.375000,0.3400
25,2021-01-03,1345834958976258048,Step 1: Resist the urge to buy Bitcoin at $32K,SatoshiLite,0.000000,0.0000
...,...,...,...,...,...,...
3153,2018-07-07,1015637151784943616,"What propaganda? Some people only care about the store of value aspect of Bitcoin. Whatever works for them. It's a decentralized currency after all. If they want to say that, that's fine. I don't fully agree. But I do support scaling transaction throughput with LN, hence the ⚡.",SatoshiLite,0.105556,0.6943
3159,2018-07-07,1015634250979086336,You have no clue how Bitcoin works. Miners care about one thing only: profit. They don't care what's in the whitepaper at all. Their job is not to enforce the whitepaper. What a joke! Can't believe you were once a Bitcoin Foundation board member. SMH,SatoshiLite,0.000000,0.1855
3165,2018-07-06,1015065815488421888,Jimmy is quitting Bitcoin dev to join Bitcoin Cash dev! 😂,SatoshiLite,0.000000,0.6588
3168,2018-07-04,1014555660984336384,"If indeed the 7000 number is correct, what happened was that many trading bots were compromise and spent all their BTC to buy SYS from the thief. So all the trading bot accounts combined has a lot of BTC.",SatoshiLite,0.200000,-0.4767


In [184]:
btc_df.reset_index(inplace=True)

In [241]:
#Grouping sentiment by date and looking at charts of sentiment and volume over time

charlie_bydate = charlie_df.groupby(['Date'], as_index=False).agg({'textblob': 'mean','vader_compound': 'mean', 'text': 'first', 'tweet_id': 'count'})
charlie_bydate

#Combining BTC price with dataframe
#charlie_bydate = charlie_bydate.set_index('Date')
btc_charlie = pd.merge(charlie_bydate,btc_df, how='inner', on= "Date")
btc_charlie.tail(10)

,Date,textblob,vader_compound,text,tweet_id,Close
260,2020-12-25,1.00,0.89,Merry Christmas &amp; Happy Holidays Litecoin Fam!!! \n\n#litecoin #litecoinfam $LTC #crypto #blockchain #bitcoin …,1,24717.00
261,2020-12-26,0.43,0.78,Community makes a crypto stronger. Some are so strong &amp; vocal that they even have made a name for themselves. There's the XRP army and Link marines. Bitcoin even has the Bitcoin maximalists.\n\nNothing beats the Litecoin Family though. Family always stick together. 👊\n\n#LitecoinFam,1,26412.00
262,2020-12-29,-0.50,-0.48,We were wrong.\n\nI’ve sold all my #Bitcoin,1,27315.00
263,2020-12-30,0.30,0.05,We don't ask that #Bitcoin be free of criticism or skepticism. We only ask for a better caliber of critic. …,1,28880.00
264,2021-01-02,0.38,0.15,By the end of this year we'll have 6 digit #Bitcoin.\n$BCH will no longer be in the top 10. \nNeither will $XRP.\nAt least 1 m…,3,32185.00
265,2021-01-03,0.40,0.29,Step 1: Resist the urge to buy Bitcoin at $32K,2,32971.00
266,2021-01-07,0.38,0.34,Excited to have $4B mcap $WBTC and $WETH launching on #TRON with \n\nMeanwhile Wrapped #Litecoin $WLTC on #TRON is…,1,39407.00
267,2021-01-14,0.17,0.00,"“The Classic Bitcoiner Response”, featuring",1,39170.00
268,2021-01-15,0.29,0.30,Here's how I got accidentally Bitcoin rich...\n\nWho wants some?,1,36741.00
269,2021-01-22,0.37,-0.82,"A more detailed explanation of the whole \n\n""A double-spend broke Bitcoin"" FUD that was circulated by an irresponsible publica…",1,33062.00


In [216]:
btc_charlie.dtypes

Date              datetime64[ns]
textblob                 float64
vader_compound           float64
text                      object
tweet_id                   int64
Close                      int32
dtype: object

In [242]:
#Creating an RESULTS dataframe for easier evaluation

#Setting up a dataframe with results
results_inf = pd.DataFrame(index=['Charlie','Roger','Andreas','Pomp','Vitalik','Combined_inf'], 
                           columns=['corr_textblob','corr_vader','btc_twt_count','avg_textblob','avg_vader'])

#getting correlation between textblob and btc price
results_inf.at['Charlie','corr_textblob'] = round(btc_charlie['Close'].corr(btc_charlie['textblob']),2)

#getting correlation between vadercompound and btc price
results_inf.at['Charlie','corr_vader'] = round(btc_charlie['Close'].corr(btc_charlie['vader_compound']),2)

#getting tweet count that talk specifically about bitcoin
results_inf.at['Charlie','btc_twt_count'] = btc_charlie['tweet_id'].sum()

#average textblob sentiment
results_inf.at['Charlie','avg_textblob'] = round(btc_charlie['textblob'].mean(),2)

#average vader sentiment
results_inf.at['Charlie','avg_vader'] = round(btc_charlie['vader_compound'].mean(),2)
results_inf

,corr_textblob,corr_vader,btc_twt_count,avg_textblob,avg_vader
Charlie,0.14,-0.03,454,0.15,0.22
Roger,NaN,NaN,NaN,NaN,NaN
Andreas,NaN,NaN,NaN,NaN,NaN
Pomp,NaN,NaN,NaN,NaN,NaN
Vitalik,NaN,NaN,NaN,NaN,NaN
Combined_inf,NaN,NaN,NaN,NaN,NaN


In [26]:
# use this for filtering tweets per date
#crypto_df[(crypto_df['Date'] > '2018-03-19') & (crypto_df['Date'] < '2018-03-21')]


In [27]:
# SECTION B: GETTING ALL TWEETS FROM @rogerkver Roger Ver 624k followers. Same code as before, just applied to a different twitter handle

screen_name = 'rogerkver'

rogertweets = tw.Cursor(api.user_timeline,
              screen_name=screen_name,
              tweet_mode='extended',
              lang =" en").items()

roger_json = []

for tweet in rogertweets:
    roger_json.append(tweet._json)


In [31]:
roger_list.clear()
roger_list

[]

In [32]:
# removing URLs, and creating DataFrame for specific tweet columns. Exporting results to csv file that can be used by itself

roger_list=[]

with open('roger.txt', 'w') as file:
        file.write(json.dumps(roger_json, indent=4))

with open('roger.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        text = clean_tweets(each_dictionary['full_text'])
        Date = each_dictionary['created_at']
        name = each_dictionary['user']['screen_name']
        
        roger_list.append({'tweet_id': str(tweet_id),
                             'text': str(text),
                             'name': str(name),
                             'Date': Date,
                            })

        roger_df = pd.DataFrame(roger_list, columns = 
                                  ['Date','tweet_id', 'text', 
                                   'name'])

roger_df.to_csv('roger.csv', index = False)

        
#pd.options.display.max_columns = None
#pd.options.display.max_colwidth = 500


In [103]:
#Reading CSV file
pd.options.display.max_colwidth = 500
roger_df = pd.read_csv('roger.csv')
roger_df

,Date,tweet_id,text,name
0,Thu Jan 21 15:13:46 +0000 2021,1352273186176593922,Do you believe in magic?\n does.,rogerkver
1,Wed Jan 20 14:51:55 +0000 2021,1351905301247774721,#BitcoinCash is the future of futures on,rogerkver
2,Tue Jan 19 18:35:38 +0000 2021,1351599213801598977,FEE is now holding millions of dollars of #BitcoinCash on their balance sheet.,rogerkver
3,Mon Jan 18 23:53:48 +0000 2021,1351316893236400136,"“If we let politicians break the law in an emergency, politicians will create an emergency so they can break the law.”",rogerkver
4,Sun Jan 17 20:47:09 +0000 2021,1350907535340011530,It’s time for the world to switch from censorable banks and to uncensorable peer to peer electronic cash and . \n\n#BitcoinCash,rogerkver
...,...,...,...,...
3201,Tue Jan 08 07:55:16 +0000 2013,288554449037848576,"""Bitcoin can protect people from their own government"" Great video on Bitcoin from John Stossel :",rogerkver
3202,Fri Jan 04 12:57:51 +0000 2013,287181045294563328,How to buy #bitcoin in your country? Try this site:,rogerkver
3203,Fri Jan 04 09:58:47 +0000 2013,287135982971670528,I just heard from a large bank in Japan. They are asking if Satoshi Nakamoto can come to their office and present on Bitcoin. #money #bank,rogerkver
3204,Wed Jan 02 03:30:32 +0000 2013,286313501776039936,"🔸Happy New Year🔸to the Three Musketeers of #Bitcoin , from d'Artagnan",rogerkver


In [104]:
# Cleaning up Date from twitter format to python readable format

roger_df = roger_df.dropna().drop_duplicates()
roger_df['Date'] = pd.to_datetime(roger_df['Date'],utc=True)
roger_df['Date'] = roger_df['Date'].dt.date
roger_df['Date'] = pd.to_datetime(roger_df['Date'])
roger_df.dtypes

# Adding textblob sentiment
roger_df['textblob']= roger_df['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
roger_df

# Adding vader sentiment
roger_df['vader_compound']= roger_df.apply(lambda row: sentiment_score_compound(row['text']), axis=1)
roger_df.drop_duplicates()

# Only getting tweets with word bitcoin
search = ['Bitcoin','bitcoin','BTC','btc','crypto','cryptocurrency']
roger_df = roger_df[roger_df['text'].str.contains('|'.join(search))]

pd.options.display.max_colwidth = 500
roger_df

,Date,tweet_id,text,name,textblob,vader_compound
1,2021-01-20,1351905301247774721,#BitcoinCash is the future of futures on,rogerkver,0.000000,0.0000
2,2021-01-19,1351599213801598977,FEE is now holding millions of dollars of #BitcoinCash on their balance sheet.,rogerkver,0.000000,0.0000
4,2021-01-17,1350907535340011530,It’s time for the world to switch from censorable banks and to uncensorable peer to peer electronic cash and . \n\n#BitcoinCash,rogerkver,0.000000,0.0000
5,2021-01-16,1350531833566527491,“The existing Visa credit card network processes about 15 million Internet purchases per day worldwide. Bitcoin can already scale much larger than that with EXISTING HARDWARE for a fraction of the cost. It never really hits a scale ceiling.” \n- Satoshi Nakamoto 2010,rogerkver,-0.050000,0.3818
6,2021-01-15,1350167596486823950,Bitcoin is stupid.,rogerkver,-0.800000,-0.5267
...,...,...,...,...,...,...
3201,2013-01-08,288554449037848576,"""Bitcoin can protect people from their own government"" Great video on Bitcoin from John Stossel :",rogerkver,0.700000,0.7717
3202,2013-01-04,287181045294563328,How to buy #bitcoin in your country? Try this site:,rogerkver,0.000000,0.0000
3203,2013-01-04,287135982971670528,I just heard from a large bank in Japan. They are asking if Satoshi Nakamoto can come to their office and present on Bitcoin. #money #bank,rogerkver,0.107143,0.0000
3204,2013-01-02,286313501776039936,"🔸Happy New Year🔸to the Three Musketeers of #Bitcoin , from d'Artagnan",rogerkver,0.136364,0.0000


In [243]:
#Grouping sentiment by date and looking at charts of sentiment and volume over time

roger_bydate = roger_df.groupby(['Date'], as_index=False).agg({'textblob': 'mean','vader_compound': 'mean', 'text': 'first', 'tweet_id': 'count'})
roger_bydate

#Combining BTC price with dataframe

roger_bydate = roger_bydate.set_index('Date')
btc_roger = pd.merge(roger_bydate,btc_df, how='inner', on= "Date")

In [244]:
#getting correlation between textblob and btc price
results_inf.at['Roger','corr_textblob'] = round(btc_roger['Close'].corr(btc_roger['textblob']),2)

#getting correlation between vadercompound and btc price
results_inf.at['Roger','corr_vader'] = round(btc_roger['Close'].corr(btc_roger['vader_compound']),2)
results_inf

#getting tweet count that talk specifically about bitcoin
results_inf.at['Roger','btc_twt_count'] = btc_roger['tweet_id'].sum()

#average textblob sentiment
results_inf.at['Roger','avg_textblob'] = round(btc_roger['textblob'].mean(),2)

#average vader sentiment
results_inf.at['Roger','avg_vader'] = round(btc_roger['vader_compound'].mean(),2)
results_inf

,corr_textblob,corr_vader,btc_twt_count,avg_textblob,avg_vader
Charlie,0.14,-0.03,454,0.15,0.22
Roger,-0.14,-0.06,501,0.14,0.19
Andreas,NaN,NaN,NaN,NaN,NaN
Pomp,NaN,NaN,NaN,NaN,NaN
Vitalik,NaN,NaN,NaN,NaN,NaN
Combined_inf,NaN,NaN,NaN,NaN,NaN


In [36]:
# SECTION C: GETTING ALL TWEETS FROM @aantonop Andreas M. Antonopoulos 534k followers. Same code as before, just applied to a different twitter handle

screen_name = 'aantonop'

aantonoptweets = tw.Cursor(api.user_timeline,
              screen_name=screen_name,
              tweet_mode='extended',
              lang =" en").items()

aantonop_json = []

for tweet in aantonoptweets:
    aantonop_json.append(tweet._json)

In [38]:
# removing URLs, and creating DataFrame for specific tweet columns

aantonop_list=[]

with open('aantonop.txt', 'w') as file:
        file.write(json.dumps(aantonop_json, indent=4))

with open('aantonop.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        text = clean_tweets(each_dictionary['full_text'])
        Date = each_dictionary['created_at']
        name = each_dictionary['user']['screen_name']
        
        aantonop_list.append({'tweet_id': str(tweet_id),
                             'text': str(text),
                             'name': str(name),
                             'Date': Date,
                            })

        aantonop_df = pd.DataFrame(aantonop_list, columns = 
                                  ['Date','tweet_id', 'text', 
                                   'name'])
        
aantonop_df.to_csv('andreas.csv', index = False)

#pd.options.display.max_columns = None
#pd.options.display.max_colwidth = 500


In [107]:
#Reading CSV file
pd.options.display.max_colwidth = 500
aantonop_df = pd.read_csv('andreas.csv')
aantonop_df

,Date,tweet_id,text,name
0,Fri Jan 22 09:02:02 +0000 2021,1352542026793934848,"In #bitcoin, privacy isn't a one and done sort of thing, but here are the simple ways you can increase #privacy on your bitcoin transactions right now:",aantonop
1,Fri Jan 22 08:15:01 +0000 2021,1352530195228581889,"Cryptocurrency Explained: Understanding ""Double-spend"", Block Re-Organization, &amp; Consensus [Bitcoin] \n#cryptocurrency #crypto #CryptocurrencyNews #bitcoin",aantonop
2,Fri Jan 22 04:32:08 +0000 2021,1352474100657123329,😆,aantonop
3,Fri Jan 22 04:26:58 +0000 2021,1352472801064542210,"Mostly yes, then piled on by other irresponsible ""reporting"" by businessinsider and other more ""mainstream"" financial press.",aantonop
4,Fri Jan 22 04:21:12 +0000 2021,1352471350997180416,The people who reported BitMEX's simple reporting of the normal operation of bitcoin in a sensationalist manner full of falsehoods. \n\nMaybe do some more research before throwing stones at the wrong target. I've written about it extensively all day.,aantonop
...,...,...,...,...
3219,Wed Jul 01 18:30:06 +0000 2020,1278395470373629953,"Hertz is trash, they routinely rent out cars without a proper pre-trip. They rented me a car with tires comple…",aantonop
3220,Wed Jul 01 18:29:56 +0000 2020,1278395431135916032,Rented from Hertz the other day. There was still a Jimmy John’s bag in the center console. Along with…,aantonop
3221,Wed Jul 01 18:21:47 +0000 2020,1278393379232112644,"No, you are not. In the state where this happened, masks are required in enclosed spaces that are open to the public. \n\nAlso, saying you sanitize cars implies you do it correctly. The fact that you don't is worse than not doing it at all. A false sense of security.",aantonop
3222,Wed Jul 01 18:17:16 +0000 2020,1278392241984241665,"Morally bankrupt, yes, like all large corporations.\n\nFinancially bankrupt? That too.",aantonop


In [108]:
# Cleaning up Date from twitter format to python readable format

aantonop_df = aantonop_df.dropna().drop_duplicates()
aantonop_df['Date'] = pd.to_datetime(aantonop_df['Date'],utc=True)
aantonop_df['Date'] = aantonop_df['Date'].dt.date
aantonop_df['Date'] = pd.to_datetime(aantonop_df['Date'])
aantonop_df.dtypes

# Adding textblob sentiment
aantonop_df['textblob']= aantonop_df['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
aantonop_df

# Adding vader sentiment
aantonop_df['vader_compound']= aantonop_df.apply(lambda row: sentiment_score_compound(row['text']), axis=1)
aantonop_df.drop_duplicates()

# Only getting tweets with word bitcoin
search = ['Bitcoin','bitcoin','BTC','btc','crypto','cryptocurrency']
aantonop_df = aantonop_df[aantonop_df['text'].str.contains('|'.join(search))]

pd.options.display.max_colwidth = 500
aantonop_df

,Date,tweet_id,text,name,textblob,vader_compound
0,2021-01-22,1352542026793934848,"In #bitcoin, privacy isn't a one and done sort of thing, but here are the simple ways you can increase #privacy on your bitcoin transactions right now:",aantonop,0.142857,0.4497
1,2021-01-22,1352530195228581889,"Cryptocurrency Explained: Understanding ""Double-spend"", Block Re-Organization, &amp; Consensus [Bitcoin] \n#cryptocurrency #crypto #CryptocurrencyNews #bitcoin",aantonop,0.000000,-0.4404
4,2021-01-22,1352471350997180416,The people who reported BitMEX's simple reporting of the normal operation of bitcoin in a sensationalist manner full of falsehoods. \n\nMaybe do some more research before throwing stones at the wrong target. I've written about it extensively all day.,aantonop,0.083333,-0.4767
8,2021-01-22,1352452964468871170,"Today there were headlines about a ""double-spend"" in #bitcoin! Many took that to mean that bitcoin has a catastrophic flaw (it doesn't) or the network is not secure (it is) or that bitcoin is somehow broken (it's not). Debunking the FUD:",aantonop,-0.103125,-0.7408
9,2021-01-22,1352438238582104065,"Curious how to run your own node for #btc, #eth, or #lightningnetwork? Join my livestream event this Saturday where I’ll be answering your questions on nodes. \n\nSubscribe to my YouTube channel &amp; set a reminder now:",aantonop,0.250000,0.5423
...,...,...,...,...,...,...
3198,2020-07-03,1279069158936915968,"New interview out on podcast. Tune in if you want to hear us discuss strengths, weaknesses, threats &amp; resilience: #Bitcoin",aantonop,0.136364,-0.3182
3199,2020-07-02,1278820218362626051,Want to know more about how #bitcoin and the #LightningNetwork work? Join me on Sunday for a live premiere of my talk from called “A Technical Introduction to the Lightning Network”. \n\nSet a reminder to watch + chat:,aantonop,0.212121,0.3612
3202,2020-07-02,1278715657178288129,"I think Transmuter could even be used to do BTC-&gt;DAI conversion, perhaps even on a DEX giving retailers permission-less decentralized stablecoin conversion\n\nMuch better than custodial solutions like BitPay which severely limit the jurisdictions and permitted activities.",aantonop,0.500000,0.6705
3206,2020-07-02,1278703518103277573,"#Bitcoin is the fundamental transformation of money. Period. Check out this #TIOMv2 highlight reel about how #bitcoin is not money for the internet, but the internet of money:",aantonop,0.000000,0.1779


In [245]:
#Grouping sentiment by date and looking at charts of sentiment and volume over time

aantonop_bydate = aantonop_df.groupby(['Date'], as_index=False).agg({'textblob': 'mean','vader_compound': 'mean', 'text': 'first', 'tweet_id': 'count'})
aantonop_bydate

#Combining BTC price with dataframe

aantonop_bydate = aantonop_bydate.set_index('Date')
btc_aantonop = pd.merge(aantonop_bydate,btc_df, how='inner', on= "Date")

In [246]:
#getting correlation between textblob and btc price
results_inf.at['Andreas','corr_textblob'] = round(btc_aantonop['Close'].corr(btc_aantonop['textblob']),2)

#getting correlation between vadercompound and btc price
results_inf.at['Andreas','corr_vader'] = round(btc_aantonop['Close'].corr(btc_aantonop['vader_compound']),2)
results_inf

#getting tweet count that talk specifically about bitcoin
results_inf.at['Andreas','btc_twt_count'] = btc_aantonop['tweet_id'].sum()

#average textblob sentiment
results_inf.at['Andreas','avg_textblob'] = round(btc_aantonop['textblob'].mean(),2)

#average vader sentiment
results_inf.at['Andreas','avg_vader'] = round(btc_aantonop['vader_compound'].mean(),2)
results_inf

,corr_textblob,corr_vader,btc_twt_count,avg_textblob,avg_vader
Charlie,0.14,-0.03,454,0.15,0.22
Roger,-0.14,-0.06,501,0.14,0.19
Andreas,0.09,-0.00,750,0.15,0.27
Pomp,NaN,NaN,NaN,NaN,NaN
Vitalik,NaN,NaN,NaN,NaN,NaN
Combined_inf,NaN,NaN,NaN,NaN,NaN


In [139]:
# SECTION D: GETTING ALL TWEETS FROM @APompliano. Same code as before, just applied to a different twitter handle

screen_name = 'APompliano'

pomptweets = tw.Cursor(api.user_timeline,
              screen_name=screen_name,
              tweet_mode='extended',
              lang =" en").items()

pomp_json = []

for tweet in pomptweets:
    pomp_json.append(tweet._json)

In [140]:
# removing URLs, and creating DataFrame for specific tweet columns

pomp_list=[]

with open('pomp.txt', 'w') as file:
        file.write(json.dumps(pomp_json, indent=4))

with open('pomp.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        text = clean_tweets(each_dictionary['full_text'])
        Date = each_dictionary['created_at']
        name = each_dictionary['user']['screen_name']
        
        pomp_list.append({'tweet_id': str(tweet_id),
                             'text': str(text),
                             'name': str(name),
                             'Date': Date,
                            })

        pomp_df = pd.DataFrame(pomp_list, columns = 
                                  ['Date','tweet_id', 'text', 
                                   'name'])

pomp_df.to_csv('pomp.csv', index = False)        
        
#pd.options.display.max_columns = None
#pd.options.display.max_colwidth = 500


In [141]:
#Reading CSV file
pd.options.display.max_colwidth = 500
pomp_df = pd.read_csv('pomp.csv')
pomp_df

,Date,tweet_id,text,name
0,Sat Jan 23 01:57:14 +0000 2021,1352797509299658752,Elon - the world is ready to hear you talk for an hour about Bitcoin. Let’s do it?,APompliano
1,Sat Jan 23 01:44:12 +0000 2021,1352794229936959488,:),APompliano
2,Sat Jan 23 01:41:56 +0000 2021,1352793657280225281,is the GOAT,APompliano
3,Sat Jan 23 01:41:43 +0000 2021,1352793603240828928,"When the UFC moved forward with live events during the pandemic, Dana White was called ""reckless"" &amp; ""irresponsible.""\n\nHis…",APompliano
4,Sat Jan 23 01:30:39 +0000 2021,1352790819292508162,This is going to be fun :),APompliano
...,...,...,...,...
3222,Wed Nov 04 05:04:59 +0000 2020,1323853727632117761,You getting old! 😂,APompliano
3223,Wed Nov 04 05:03:35 +0000 2020,1323853377818763264,Disappointing that cable networks won’t report numbers,APompliano
3224,Wed Nov 04 04:59:28 +0000 2020,1323852341645365250,:),APompliano
3225,Wed Nov 04 04:25:18 +0000 2020,1323843742814654467,[NEW POST] How Investors Are Preparing For Election 2020\n\nI spoke with the smartest investors I know and here is how they a…,APompliano


In [142]:
# Cleaning up Date from twitter format to python readable format

pomp_df = pomp_df.dropna().drop_duplicates()
pomp_df['Date'] = pd.to_datetime(pomp_df['Date'],utc=True)
pomp_df['Date'] = pomp_df['Date'].dt.date
pomp_df['Date'] = pd.to_datetime(pomp_df['Date'])
pomp_df.dtypes

# Adding textblob sentiment
pomp_df['textblob']= pomp_df['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
pomp_df

# Adding vader sentiment
pomp_df['vader_compound']= pomp_df.apply(lambda row: sentiment_score_compound(row['text']), axis=1)
pomp_df.drop_duplicates()

# Only getting tweets with word bitcoin
search = ['Bitcoin','bitcoin','BTC','btc','crypto','cryptocurrency']
pomp_df = pomp_df[pomp_df['text'].str.contains('|'.join(search))]

pd.options.display.max_colwidth = 500
pomp_df

,Date,tweet_id,text,name,textblob,vader_compound
0,2021-01-23,1352797509299658752,Elon - the world is ready to hear you talk for an hour about Bitcoin. Let’s do it?,APompliano,0.200000,0.3612
7,2021-01-23,1352769980056694784,"Every morning I write a letter to 115,000 investors and break down Bitcoin, business, &amp; finance in simple terms.\n\nSubscribers include billionaires, large institutional allocators, &amp; world’s most successful investors.\n\nRead what the smart money is reading:",APompliano,0.253836,0.7778
12,2021-01-22,1352761086618071040,"One of the smartest, but I am very surprised by Mike's position on Bitcoin. I think we can help change that!",APompliano,0.162500,0.8455
15,2021-01-22,1352760096674226185,I hereby challenge to a debate on Bitcoin and the future prospects of the digital currency.\n\nHe can pick the venue and the moderator.\n\nI'll be there as long as he isn't afraid of changing his mind :),APompliano,-0.030000,0.6705
16,2021-01-22,1352755440329908227,. is launching a debit card that will allow you to spend crypto anywhere that Mastercard is accepted.,APompliano,0.000000,0.4588
...,...,...,...,...,...,...
3195,2020-11-05,1324332874733096960,Bitcoin is up 107% in 2020.,APompliano,0.000000,0.0000
3198,2020-11-05,1324329549111398402,Good morning to everyone except the people who said the Bitcoin halving was priced in.,APompliano,0.700000,0.4404
3209,2020-11-04,1324047969759436801,"This makes no sense. \n\nYou would to have bought Bitcoin on one of only about 40 days in the last 11 years to be unprofitable if you held it till today.\n\nThe smart money is buying Bitcoin right now. Unless you think Paul Tudor Jones, Fidelity, Square, and others are not smart?",APompliano,0.078571,-0.1921
3211,2020-11-04,1324041914707005446,Bitcoin is up 98% in 2020.,APompliano,0.000000,0.0000


In [247]:
#Grouping sentiment by date and looking at charts of sentiment and volume over time

pomp_bydate = pomp_df.groupby(['Date'], as_index=False).agg({'textblob': 'mean','vader_compound': 'mean', 'text': 'first', 'tweet_id': 'count'})
pomp_bydate

#Combining BTC price with dataframe

pomp_bydate = pomp_bydate.set_index('Date')
btc_pomp = pd.merge(pomp_bydate,btc_df, how='inner', on= "Date")

In [248]:
#getting correlation between textblob and btc price
results_inf.at['Pomp','corr_textblob'] = round(btc_pomp['Close'].corr(btc_pomp['textblob']),2)

#getting correlation between vadercompound and btc price
results_inf.at['Pomp','corr_vader'] = round(btc_pomp['Close'].corr(btc_pomp['vader_compound']),2)
results_inf

#getting tweet count that talk specifically about bitcoin
results_inf.at['Pomp','btc_twt_count'] = btc_pomp['tweet_id'].sum()

#average textblob sentiment
results_inf.at['Pomp','avg_textblob'] = round(btc_pomp['textblob'].mean(),2)

#average vader sentiment
results_inf.at['Pomp','avg_vader'] = round(btc_pomp['vader_compound'].mean(),2)
results_inf

,corr_textblob,corr_vader,btc_twt_count,avg_textblob,avg_vader
Charlie,0.14,-0.03,454,0.15,0.22
Roger,-0.14,-0.06,501,0.14,0.19
Andreas,0.09,-0.00,750,0.15,0.27
Pomp,-0.18,0.15,535,0.18,0.26
Vitalik,NaN,NaN,NaN,NaN,NaN
Combined_inf,NaN,NaN,NaN,NaN,NaN


In [43]:
# SECTION D: GETTING ALL TWEETS FROM @VitalikButerin. Same code as before, just applied to a different twitter handle

screen_name = 'VitalikButerin'

vitaliktweets = tw.Cursor(api.user_timeline,
              screen_name=screen_name,
              tweet_mode='extended',
              lang =" en").items()

vitalik_json = []

for tweet in vitaliktweets:
    vitalik_json.append(tweet._json)

In [44]:
# removing URLs, and creating DataFrame for specific tweet columns

vitalik_list=[]

with open('vitalik.txt', 'w') as file:
        file.write(json.dumps(vitalik_json, indent=4))

with open('vitalik.txt', encoding='utf-8') as json_file:  
    all_data = json.load(json_file)
    for each_dictionary in all_data:
        tweet_id = each_dictionary['id']
        text = clean_tweets(each_dictionary['full_text'])
        Date = each_dictionary['created_at']
        name = each_dictionary['user']['screen_name']
        
        vitalik_list.append({'tweet_id': str(tweet_id),
                             'text': str(text),
                             'name': str(name),
                             'Date': Date,
                            })

        vitalik_df = pd.DataFrame(vitalik_list, columns = 
                                  ['Date','tweet_id', 'text', 
                                   'name'])

vitalik_df.to_csv('vitalik.csv', index = False)        
        
#pd.options.display.max_columns = None
#pd.options.display.max_colwidth = 500


In [111]:
#Reading CSV file
pd.options.display.max_colwidth = 500
vitalik_df = pd.read_csv('vitalik.csv')
vitalik_df

,Date,tweet_id,text,name
0,Tue Jan 12 07:28:04 +0000 2021,1348894498856660992,"This is a dangerous moment in a lot of ways, but there's a lot of work to be done!",VitalikButerin
1,Tue Jan 12 07:28:03 +0000 2021,1348894495329320960,"Another is to try some cool ideas at combining anonymity with persistent reputations, eg. see here: \n\nBasically get the benefits of privacy while still keeping a way to hold people accountable, in zero knowledge.",VitalikButerin
2,Tue Jan 12 07:28:02 +0000 2021,1348894491793494018,The crypto space can also participate here. is a great experiment. Another cool experiment would be to try decentralized forums moderated by Kleros. Kleros has already shown its sanity by correctly deciding that Biden has won the election:,VitalikButerin
3,Tue Jan 12 07:28:01 +0000 2021,1348894488610000897,"I credit for experimenting here already: tweets with limits on who can reply to them were a great idea. The ""quote-tweet-by-default"" experiment was a nice try. I wonder what else we can do. Or maybe we just need less tweeting and more long-form (as I am doing in 2021).",VitalikButerin
4,Tue Jan 12 07:28:01 +0000 2021,1348894485552369664,"But clearly GIFT is not so simple. Plenty of online interactions are civil, and plenty of non-anonymous interactions are not. Are there ways to tweak the ways that we interact with the internet to make us more pro-social? I think so!",VitalikButerin
...,...,...,...,...
3237,Thu Jun 11 11:29:38 +0000 2020,1271041901765054466,"&gt; There's also the possibility of a cartel who don't fork out blocks over 10M gas, but instead refuse to produce them. There'd be free riders, but I haven't seen anyone analyse it.\n\nA quick explanation of why this isn't an issue is in the FAQ:",VitalikButerin
3238,Thu Jun 11 11:21:44 +0000 2020,1271039913069289472,"&gt; The assumption of 50% seems naive, too. Isn't the actual lower bound for a 51% attack closer to 33%?\n\nWhy so? Even including uncle rate issues, you still need ~46% to do a 51% attack right now.",VitalikButerin
3239,Thu Jun 11 00:31:29 +0000 2020,1270876269744439296,Why would you want to defund it? Haven't you heard that paw patrol is literally neoliberal bootlicker capitalist propaganda?\n\n,VitalikButerin
3240,Wed Jun 10 21:27:59 +0000 2020,1270830091124109312,"I think it originated on /r/buttcoin.\n\nAs I remember the vocab there:\n\nbitcoin -&gt; buttcoin\nETH -&gt; mETH (or ""etherbutts"", or ""ethere-ho-hum"")\nIOTA -&gt; IdIOTA\nETC -&gt; classic mETH\nme -&gt; Vitalik Butterin (two t's)",VitalikButerin


In [112]:
# Cleaning up Date from twitter format to python readable format

vitalik_df = vitalik_df.dropna().drop_duplicates()
vitalik_df['Date'] = pd.to_datetime(vitalik_df['Date'],utc=True)
vitalik_df['Date'] = vitalik_df['Date'].dt.date
vitalik_df['Date'] = pd.to_datetime(vitalik_df['Date'])
vitalik_df.dtypes

# Adding textblob sentiment
vitalik_df['textblob']= vitalik_df['text'].apply(lambda tweet: TextBlob(tweet).sentiment.polarity)
vitalik_df

# Adding vader sentiment
vitalik_df['vader_compound']= vitalik_df.apply(lambda row: sentiment_score_compound(row['text']), axis=1)
vitalik_df.drop_duplicates()

# Only getting tweets with word bitcoin
search = ['Bitcoin','bitcoin','BTC','btc','crypto','cryptocurrency']
vitalik_df = vitalik_df[vitalik_df['text'].str.contains('|'.join(search))]

pd.options.display.max_colwidth = 500
vitalik_df

,Date,tweet_id,text,name,textblob,vader_compound
2,2021-01-12,1348894491793494018,The crypto space can also participate here. is a great experiment. Another cool experiment would be to try decentralized forums moderated by Kleros. Kleros has already shown its sanity by correctly deciding that Biden has won the election:,VitalikButerin,0.575000,0.8779
65,2020-12-28,1343374967589629952,"Also, I hold more in fiat than I do in bitcoin, so if you think hedging is a form of kneeling to the king, you may want to think about which king I am kneeling to. 🤣",VitalikButerin,0.500000,0.5423
76,2020-12-22,1341219519545556992,"Looks like the Ripple/XRP team is sinking to new levels of strangeness. They're claiming that their shitcoin should not be called a security for *public policy reasons*, namely because Bitcoin and Ethereum are ""Chinese-controlled"". 😂😂\n\n",VitalikButerin,0.068182,0.8658
97,2020-12-14,1338394465954754560,"7 years ago, before ethereum even began, I had only a few thousand dollars of net worth. I nevertheless sold half my bitcoin to make sure that I would not be broke if BTC went to zero.",VitalikButerin,0.072222,0.6738
100,2020-12-14,1338391068073578497,Isn't the microstrategy thing just them trying to turn their company into a de-facto bitcoin ETF for regulatory arbitrage purposes?,VitalikButerin,0.000000,0.0000
...,...,...,...,...,...,...
3198,2020-06-12,1271447150023630848,So I guess what are you looking for that you don't have already?\n\nAn actually decentralized crypto &lt;-&gt; fiat platform similar to ? Something else?,VitalikButerin,0.000000,0.0000
3199,2020-06-12,1271445153870471169,What we expected: cryptocurrency would normalize and become more like the stock market\nWhat happened: the outside world went crazy and the stock market became more like cryptocurrency,VitalikButerin,0.060000,0.4915
3213,2020-06-12,1271415981504675840,"Oh I think bitcoin NG is quite overrated. The optimistic analysis basically assumes that uncle rate is the only constraint on block size, but in reality it's one of like five (the others being state growth, DoS risk, sync time, ease of running a full node...)",VitalikButerin,0.175000,0.7953
3226,2020-06-11,1271163110070632448,The Ontario Securities Commission says Vancouver-based cryptocurrency exchange Quadriga collapsed because of fraud committed…,VitalikButerin,0.000000,-0.5719


In [249]:
#Grouping sentiment by date and looking at charts of sentiment and volume over time

vitalik_bydate = vitalik_df.groupby(['Date'], as_index=False).agg({'textblob': 'mean','vader_compound': 'mean', 'text': 'first', 'tweet_id': 'count'})
vitalik_bydate

#Combining BTC price with dataframe

vitalik_bydate = vitalik_bydate.set_index('Date')
btc_vitalik = pd.merge(vitalik_bydate,btc_df, how='inner', on= "Date")

In [250]:
#getting correlation between textblob and btc price
results_inf.at['Vitalik','corr_textblob'] = round(btc_vitalik['Close'].corr(btc_vitalik['textblob']),2)

#getting correlation between vadercompound and btc price
results_inf.at['Vitalik','corr_vader'] = round(btc_vitalik['Close'].corr(btc_vitalik['vader_compound']),2)
results_inf

#getting tweet count that talk specifically about bitcoin
results_inf.at['Vitalik','btc_twt_count'] = btc_vitalik['tweet_id'].sum()

#average textblob sentiment
results_inf.at['Vitalik','avg_textblob'] = round(btc_vitalik['textblob'].mean(),2)

#average vader sentiment
results_inf.at['Vitalik','avg_vader'] = round(btc_vitalik['vader_compound'].mean(),2)
results_inf

,corr_textblob,corr_vader,btc_twt_count,avg_textblob,avg_vader
Charlie,0.14,-0.03,454,0.15,0.22
Roger,-0.14,-0.06,501,0.14,0.19
Andreas,0.09,-0.00,750,0.15,0.27
Pomp,-0.18,0.15,535,0.18,0.26
Vitalik,0.26,0.25,212,0.13,0.21
Combined_inf,NaN,NaN,NaN,NaN,NaN


In [251]:
#Combining the  dataframes together.

combined_influencers = [vitalik_df,aantonop_df, roger_df,charlie_df]  
combined_inf_df = pd.concat(combined_influencers, axis=0, ignore_index=True)

In [252]:
combined_inf_df = combined_inf_df.sort_values('Date', ascending = False)
combined_inf_df

,Date,tweet_id,text,name,textblob,vader_compound
213,2021-01-22,1352530195228581889,"Cryptocurrency Explained: Understanding ""Double-spend"", Block Re-Organization, &amp; Consensus [Bitcoin] \n#cryptocurrency #crypto #CryptocurrencyNews #bitcoin",aantonop,0.00,-0.44
216,2021-01-22,1352438238582104065,"Curious how to run your own node for #btc, #eth, or #lightningnetwork? Join my livestream event this Saturday where I’ll be answering your questions on nodes. \n\nSubscribe to my YouTube channel &amp; set a reminder now:",aantonop,0.25,0.54
212,2021-01-22,1352542026793934848,"In #bitcoin, privacy isn't a one and done sort of thing, but here are the simple ways you can increase #privacy on your bitcoin transactions right now:",aantonop,0.14,0.45
217,2021-01-22,1352424786576633858,I'm not a maxi. \n\nI'm also a coder and wrote the #1 technical college level book about Bitcoin in 2014. \n\nPerhaps consider my work and read what I actually said. I think you will find it interesting,aantonop,0.17,0.40
2728,2021-01-22,1352478133224493059,"A more detailed explanation of the whole \n\n""A double-spend broke Bitcoin"" FUD that was circulated by an irresponsible publica…",SatoshiLite,0.37,-0.82
...,...,...,...,...,...,...
2723,2013-01-08,288554449037848576,"""Bitcoin can protect people from their own government"" Great video on Bitcoin from John Stossel :",rogerkver,0.70,0.77
2724,2013-01-04,287181045294563328,How to buy #bitcoin in your country? Try this site:,rogerkver,0.00,0.00
2725,2013-01-04,287135982971670528,I just heard from a large bank in Japan. They are asking if Satoshi Nakamoto can come to their office and present on Bitcoin. #money #bank,rogerkver,0.11,0.00
2726,2013-01-02,286313501776039936,"🔸Happy New Year🔸to the Three Musketeers of #Bitcoin , from d'Artagnan",rogerkver,0.14,0.00


In [253]:
#Grouping sentiment by date and looking at charts of sentiment and volume over time

combined_inf_bydate = combined_inf_df.groupby(['Date'], as_index=False).agg({'textblob': 'mean','vader_compound': 'mean', 'text': 'first', 'tweet_id': 'count'})
combined_inf_bydate

#Combining BTC price with dataframe
#combined_inf_bydate = combined_inf_bydate.set_index('Date')
btc_combined_inf = pd.merge(combined_inf_bydate,btc_df, how='inner', on= "Date")
btc_combined_inf

#exporting dataframe to csv file
btc_combined_inf.to_csv('inf_sentiment_btc.csv', index = False) 


In [254]:

#getting correlation between textblob and btc price
results_inf.at['Combined_inf','corr_textblob'] = round(btc_combined_inf['Close'].corr(btc_combined_inf['textblob']),2)

#getting correlation between vadercompound and btc price
results_inf.at['Combined_inf','corr_vader'] = round(btc_combined_inf['Close'].corr(btc_combined_inf['vader_compound']),2)

#getting tweet count that talk specifically about bitcoin
results_inf.at['Combined_inf','btc_twt_count'] = results_inf.iloc[0:3,2].sum(axis=0)

#average textblob sentiment
results_inf.at['Combined_inf','avg_textblob'] = round(btc_combined_inf['textblob'].mean(),2)

#average vader sentiment
results_inf.at['Combined_inf','avg_vader'] = round(btc_combined_inf['vader_compound'].mean(),2)

results_inf

,corr_textblob,corr_vader,btc_twt_count,avg_textblob,avg_vader
Charlie,0.14,-0.03,454,0.15,0.22
Roger,-0.14,-0.06,501,0.14,0.19
Andreas,0.09,-0.00,750,0.15,0.27
Pomp,-0.18,0.15,535,0.18,0.26
Vitalik,0.26,0.25,212,0.13,0.21
Combined_inf,0.02,0.02,1705,0.15,0.22


In [255]:
#exporting dataframe to csv file
results_inf.to_csv('inf_results.csv', index = True)